# Tuning

Importazione librerie

In [2]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import preprocess_dataset as ut
from tfkan import DenseKAN
from keras_tuner import HyperModel, RandomSearch, Hyperband
from keras_tuner.tuners import BayesianOptimization
import tensorflow as tf
from skopt import BayesSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import StratifiedKFold
from skopt.callbacks import DeadlineStopper
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import tensorflow as tf
from tensorflow import keras
from keras import layers, regularizers
from keras_tuner import RandomSearch, HyperModel
from sklearn.model_selection import KFold
from sklearn.utils import check_random_state


Impostiamo il random state

In [3]:
# Valore del seme causale
seed_value = 0

# Impostazione dei semi casuali per os, random, numpy e tensorflow
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Controllo che il seme sia stato correttamente impostato
check_random_state(seed_value)

RandomState(MT19937) at 0x23567A72840

Prepariamo il dataset 

In [4]:
x_train = pd.read_csv("datasets/x_train.csv")   # Caricamento del dataset
y_train = pd.read_csv("datasets/y_train.csv")
x_test = pd.read_csv("datasets/x_test.csv")  
y_test = pd.read_csv("datasets/y_test.csv")

ut.standardize(x_train)                         # Standardizzazione
x_train = x_train.to_numpy()[:, 1:-1]
y_train = y_train.to_numpy()[:, -1]
ut.standardize(x_test)                   
x_test = x_test.to_numpy()[:, 1:-1]
y_test = y_test.to_numpy()[:, -1]

x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)   # Conversione a tensore
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)  
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)


## Tuner architettura KAN

In [5]:
class HyperKAN(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = keras.Sequential()
        model.add(tf.keras.layers.Input(shape=self.input_shape))

        num_layers = hp.Int('num_layers', 1, 3)
        for i in range(num_layers):
            grid_range = hp.Float(f'grid_range_min_{i}', 1.0, 4.0)
            model.add(DenseKAN(
                units=hp.Int(f'units_{i}', 1, 16),
                grid_size=hp.Int(f'grid_size_{i}', 8, 32),
                grid_range=[
                    -grid_range,
                    grid_range
                ]
            ))
        
        model.add(DenseKAN(1))

        learning_rate = hp.Float('learning_rate', 1e-4, 1e-1, sampling='log')
        optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'adadelta', 'adagrad'])
        
        opt = tf.keras.optimizers.get(optimizer)
        opt.learning_rate = learning_rate

        model.compile(
            optimizer=opt,
            loss='mean_absolute_error',
            metrics=[tf.keras.metrics.MeanAbsoluteError(name='mae')]
        )
        return model


def run_tuner(x_train, y_train, x_test, y_test, input_shape):
    hypermodel = HyperKAN(input_shape=input_shape)

    tuner = Hyperband(
        hypermodel,
        objective='val_loss',
        max_epochs=50,
        directory="./modelli_salvati",
        project_name='retina_kan'
    )

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    kfold = KFold(n_splits=5, shuffle=True, random_state=0)

    x_train = tf.convert_to_tensor(x_train)
    y_train = tf.convert_to_tensor(y_train)

    for fold, (train_indices, val_indices) in enumerate(kfold.split(x_train.numpy())):
        print(f"Fold {fold + 1}")
        
        x_train_fold = tf.gather(x_train, train_indices)
        x_val_fold = tf.gather(x_train, val_indices)
        y_train_fold = tf.gather(y_train, train_indices)
        y_val_fold = tf.gather(y_train, val_indices)

        tuner.search(
            x_train_fold, y_train_fold,
            epochs=100,
            validation_data=(x_val_fold, y_val_fold),
            callbacks=[early_stopping]
        )

    tuner.results_summary()
    best_model = tuner.get_best_models(num_models=1)[0]

    test_loss, test_mae = best_model.evaluate(x_test, y_test)
    print(f"Test MAE: {test_mae}")

    best_model.save('best_retina_kan_model.h5')

    return best_model

N_FEATURES = x_train.shape[1]
best_model = run_tuner(x_train, y_train, x_test, y_test, input_shape=(N_FEATURES,))

Reloading Tuner from ./modelli_salvati\retina_kan\tuner0.json
Fold 1

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
2                 |3                 |num_layers
2.0172            |2.4101            |grid_range_min_0
8                 |14                |units_0
30                |23                |grid_size_0
0.00066398        |0.0016175         |learning_rate
adagrad           |rmsprop           |optimizer
2.4048            |1                 |grid_range_min_1
16                |1                 |units_1
13                |8                 |grid_size_1
2.3714            |1                 |grid_range_min_2
4                 |1                 |units_2
23                |8                 |grid_size_2
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
3                 |3                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


KeyboardInterrupt: 

## Tuner architettura MLP

In [7]:
class MyHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = keras.Sequential()
        model.add(tf.keras.layers.Input(shape=self.input_shape))

        num_layers = hp.Int('num_layers', 1, 5)
        for i in range(num_layers):
            model.add(layers.Dense(
                units=hp.Int(f'units_{i}', 4, 256, step=4),
                activation=hp.Choice(f'activation_{i}', ['relu', 'elu', 'selu']),
                kernel_regularizer=regularizers.l2(hp.Float(f'l2_{i}', 1e-4, 1e-2, sampling='log'))
            ))

            use_dropout = hp.Boolean(f'use_dropout_{i}')
            if use_dropout:
                model.add(layers.Dropout(rate=0.5))

        
        model.add(layers.Dense(1))

        learning_rate = hp.Float('learning_rate', 1e-4, 1e-1, sampling='log')
        optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'adadelta', 'adagrad'])

        opt = tf.keras.optimizers.get(optimizer)
        opt.learning_rate = learning_rate

        model.compile(
            optimizer=opt,
            loss='mean_absolute_error',
            metrics=[tf.keras.metrics.MeanAbsoluteError(name='mae')]
        )
        return model


def run_tuner(x_train, y_train, x_test, y_test, input_shape):
    hypermodel = MyHyperModel(input_shape=input_shape)

    tuner = Hyperband(
        hypermodel,
        objective='val_loss',
        max_epochs=50,
        directory="./modelli_salvati",
        project_name='retina_mlp',

    )

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    kfold = KFold(n_splits=5, shuffle=True, random_state=0)

    x_train = tf.convert_to_tensor(x_train)
    y_train = tf.convert_to_tensor(y_train)

    for fold, (train_indices, val_indices) in enumerate(kfold.split(x_train.numpy())):
        print(f"Fold {fold + 1}")
        
        x_train_fold = tf.gather(x_train, train_indices)
        x_val_fold = tf.gather(x_train, val_indices)
        y_train_fold = tf.gather(y_train, train_indices)
        y_val_fold = tf.gather(y_train, val_indices)

        tuner.search(
            x_train_fold, y_train_fold,
            epochs=50,
            validation_data=(x_val_fold, y_val_fold),
            callbacks=[early_stopping]
        )

    tuner.results_summary()
    best_model = tuner.get_best_models(num_models=1)[0]

    test_loss, test_mae = best_model.evaluate(x_test, y_test)
    print(f"Test MAE: {test_mae}")

    best_model.save('best_retina_kan_model.h5')

    return best_model

# Assicurati che x_train, y_train, x_test, y_test siano definiti correttamente
N_FEATURES = x_train.shape[1]
best_model = run_tuner(x_train, y_train, x_test, y_test, input_shape=(N_FEATURES,))


Trial 10 Complete [00h 00m 04s]
val_loss: 0.8452697396278381

Best val_loss So Far: 0.769001305103302
Total elapsed time: 00h 00m 42s

Search: Running Trial #11

Value             |Best Value So Far |Hyperparameter
3                 |3                 |num_layers
52                |112               |units_0
selu              |relu              |activation_0
0.0050511         |0.0010242         |l2_0
False             |True              |use_dropout_0
0.0011106         |0.0006935         |learning_rate
adam              |adam              |optimizer
84                |108               |units_1
selu              |relu              |activation_1
0.00025579        |0.0015578         |l2_1
True              |True              |use_dropout_1
244               |28                |units_2
relu              |selu              |activation_2
0.001661          |0.00018398        |l2_2
True              |False             |use_dropout_2
28                |8                 |units_3
selu          

KeyboardInterrupt: 

# Tuning modelli Machine Learning
Tutti i punteggi seguono la convenzione secondo cui valori restituiti più alti sono migliori di valori restituiti più bassi. Pertanto le metriche che misurano la distanza tra il modello e i dati, come metrics.mean_squared_error, sono disponibili come neg_mean_squared_error che restituiscono il valore negato della metrica.

## Tuning albero di regressione

In [5]:
import numpy as np
import tensorflow as tf
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import StratifiedKFold
from skopt import BayesSearchCV
from skopt.space import Integer, Real

class TensorToNumpyTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.numpy() if isinstance(X, tf.Tensor) else X

def create_preprocessor():
    return Pipeline([
        ('to_numpy', TensorToNumpyTransformer()),
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', RobustScaler())
    ])

def configure_search(estimator, param_space, n_iter=200, cv=3, n_jobs=-1):
    return BayesSearchCV(
        estimator=estimator,
        search_spaces=param_space,
        n_iter=n_iter,
        cv=StratifiedKFold(n_splits=cv, shuffle=True, random_state=42),
        n_jobs=n_jobs,
        return_train_score=True,
        scoring='neg_mean_squared_error',  # Changed to single scoring metric
        verbose=1,  # Add verbosity to see progress
        optimizer_kwargs={'base_estimator': 'GP'},
        random_state=0
    )

tree_param_space = {
    'regressor__max_depth': Integer(2, 50),
    'regressor__ccp_alpha': Real(0.0, 1.0),
    'regressor__min_samples_split': Integer(2, 20),
    'regressor__min_samples_leaf': Integer(1, 10),
}

def create_pipeline(estimator):
    return Pipeline([
        ('preprocessor', create_preprocessor()),
        ('regressor', estimator)
    ])

# Assume x_train and y_train are your TensorFlow tensors
x_train_np = x_train.numpy() if isinstance(x_train, tf.Tensor) else x_train
y_train_np = y_train.numpy() if isinstance(y_train, tf.Tensor) else y_train

# Ensure y is 1D
y_train_np = np.ravel(y_train_np)

tree_pipeline = create_pipeline(DecisionTreeRegressor(random_state=42))
tree_search = configure_search(tree_pipeline, tree_param_space)

print("Tuning DecisionTreeRegressor...")
tree_search.fit(x_train_np, y_train_np)

print(f"Best parameters for DecisionTreeRegressor: {tree_search.best_params_}")
print(f"Best negative mean squared error: {tree_search.best_score_:.4f}")
print("Top 5 configurations:")
results_df = pd.DataFrame(tree_search.cv_results_)
top_5 = results_df.nlargest(5, 'mean_test_score')
print(top_5[['params', 'mean_test_score', 'std_test_score']])

Tuning DecisionTreeRegressor...
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each

c:\Users\gobba\miniconda3\envs\kambinete\lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [0.0, 35, 10, 20] before, using random point [0.14519028705451967, 49, 3, 16]
  warnings.warn(


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

## Tuning foresta di regressione

In [7]:
import tensorflow as tf
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical

class TensorToNumpyTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.numpy() if isinstance(X, tf.Tensor) else X

def create_preprocessor():
    return Pipeline([
        ('to_numpy', TensorToNumpyTransformer()),
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', RobustScaler())
    ])

def configure_search(estimator, param_space, n_iter=75, cv=3, n_jobs=-1):
    return BayesSearchCV(
        estimator=estimator,
        search_spaces=param_space,
        n_iter=n_iter,
        cv=StratifiedKFold(n_splits=cv, shuffle=True, random_state=42),
        n_jobs=n_jobs,
        return_train_score=True,
        scoring=['neg_mean_squared_error', 'r2'],
        refit='neg_mean_squared_error',
        optimizer_kwargs={'base_estimator': 'GP'},
        random_state=0
    )

forest_param_space = {
    'regressor__n_estimators': Integer(10, 200),
    'regressor__max_depth': Integer(2, 50),
    'regressor__min_samples_split': Integer(2, 20),
    'regressor__min_samples_leaf': Integer(1, 10),
    'regressor__max_features': Categorical(['sqrt', 'log2', None]),
    'regressor__bootstrap': Categorical([True, False]),
}

def create_pipeline(estimator):
    return Pipeline([
        ('preprocessor', create_preprocessor()),
        ('regressor', estimator)
    ])

# Assume x_train and y_train are your TensorFlow tensors
x_train_np = x_train.numpy() if isinstance(x_train, tf.Tensor) else x_train
y_train_np = y_train.numpy() if isinstance(y_train, tf.Tensor) else y_train

forest_pipeline = create_pipeline(RandomForestRegressor(random_state=0))
forest_search = configure_search(forest_pipeline, forest_param_space)

print("Tuning RandomForestRegressor...")
forest_search.fit(x_train_np, y_train_np)
print(f"Best parameters for RandomForestRegressor: {forest_search.best_params_}")
print(f"Best negative mean squared error: {forest_search.best_score_['neg_mean_squared_error']:.4f}")
print(f"Best R-squared score: {forest_search.best_score_['r2']:.4f}\n")

Tuning RandomForestRegressor...


KeyboardInterrupt: 

## Tuning voting regressor

In [19]:
import tensorflow as tf
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import StratifiedKFold
from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical

class TensorToNumpyTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.numpy() if isinstance(X, tf.Tensor) else X

def create_preprocessor():
    return Pipeline([
        ('to_numpy', TensorToNumpyTransformer()),
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', RobustScaler())
    ])

class CustomBayesSearchCV(BayesSearchCV):
    def _run_search(self, evaluate_candidates):
        """Override the method to handle custom parameter transformation."""
        param_grid = self.search_spaces
        for i in range(self.n_iter):
            # Generate random parameters from the space
            sampled_params = {k: v.rvs(1)[0] for k, v in param_grid.items()}
            # Manually combine weights
            weights = [sampled_params.pop('weights_0'), sampled_params.pop('weights_1'), sampled_params.pop('weights_2')]
            sampled_params['weights'] = weights
            # Evaluate candidates with transformed parameters
            evaluate_candidates([sampled_params])

voting_param_space = {
    'tree__max_depth': Integer(2, 50),
    'tree__ccp_alpha': Real(0.0, 1.0),
    'tree__min_samples_split': Integer(2, 20),
    'svr__C': Real(0.1, 100, prior='log-uniform'),
    'svr__epsilon': Real(0.01, 1.0, prior='log-uniform'),
    'svr__kernel': Categorical(['linear', 'rbf', 'poly']),
    'svr__gamma': Real(1e-4, 1, prior='log-uniform'),
    'weights_0': Real(0, 1, prior='uniform'),
    'weights_1': Real(0, 1, prior='uniform'),
    'weights_2': Real(0, 1, prior='uniform'),
}

# Assume x_train and y_train are your TensorFlow tensors
x_train_np = x_train.numpy() if isinstance(x_train, tf.Tensor) else x_train
y_train_np = y_train.numpy() if isinstance(y_train, tf.Tensor) else y_train

voting_regressor = VotingRegressor(
    estimators=[
        ('tree', DecisionTreeRegressor(random_state=42)),
        ('svr', SVR()),
        ('bayes', BayesianRidge())
    ],
    weights=[1, 1, 1]  # Initialize with equal weights
)

voting_search = CustomBayesSearchCV(
    estimator=voting_regressor,
    search_spaces=voting_param_space,
    n_iter=1,  # Specify the number of iterations you need
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
    n_jobs=-1,
    return_train_score=True,
    scoring=['neg_mean_squared_error', 'r2'],
    refit='neg_mean_squared_error',
    optimizer_kwargs={'base_estimator': 'GP'},
    random_state=0
)

print("Tuning VotingRegressor...")
voting_search.fit(x_train_np, y_train_np)
print(f"Best parameters for VotingRegressor: {voting_search.best_params_}")
print(f"Best negative mean squared error: {voting_search.best_score_['neg_mean_squared_error']:.4f}")
print(f"Best R-squared score: {voting_search.best_score_['r2']:.4f}\n")

Tuning VotingRegressor...
Best parameters for VotingRegressor: {'tree__max_depth': 2, 'tree__ccp_alpha': 0.018789800436355145, 'tree__min_samples_split': 7, 'svr__C': 6.859416411328702, 'svr__epsilon': 0.1713436419711966, 'svr__kernel': 'poly', 'svr__gamma': 0.05336803302756715, 'weights': [0.35950790057378607, 0.43703195379934157, 0.697631195927265]}


IndexError: invalid index to scalar variable.